In [2]:
import pandas as pd
import os
import datetime
import math
import time
import numpy as np
from operator import itemgetter
file_path = "csv/"
current_time = datetime.datetime.now()
#print (current_time.minute)

In [9]:
def load_csvs(os_path):
    '''file_pointer = math.ceil(time_of_day.minute / 15)
    if file_pointer == 0:
        file_pointer += 1'''
    csv_paths = [f for f in os.listdir(os_path+'/') if not f.startswith('.')]
    print (csv_paths)
    
    #load_files
    clicks_df = pd.read_csv(csv_path+'clicks'+'_'+str(file_pointer)+'.csv')
    impressions_df = pd.read_csv(csv_path+'impressions'+'_'+str(file_pointer)+'.csv')
    conversions_df = pd.read_csv(csv_path+'conversions'+'_'+str(file_pointer)+'.csv')
    
    #drop_duplicates
    clicks_df = clicks_df.drop_duplicates()
    impressions_df = impressions_df.drop_duplicates()
    conversions_df = conversions_df.drop_duplicates()
    
    
    #join_dataframes_on_key_columns
    clicks_and_impressions = pd.merge(clicks_df, impressions_df, on=['banner_id','campaign_id'], how = 'left')
    merged_df = pd.merge(clicks_and_impressions, conversions_df, on = ['click_id'], how ='left')
    #print (merged_df.loc[merged_df['campaign_id']==27])
    return merged_df

merged_df = load_csvs(file_path)
merged_df.head(5)

['1', '4', '3', '2']


NameError: name 'csv_path' is not defined

In [175]:
def find_banners_with_conv(campaign_id, merged_df):
    banner_rev_dict = {}
    banner_clicks_dict = {}
    banner_ids = merged_df.loc[merged_df['campaign_id'] == campaign_id]['banner_id'].unique()
    #print (banner_ids)
    for bnr_ids in banner_ids:
        revenues_per_bnr = merged_df.loc[(merged_df['banner_id'] == bnr_ids) & (
            merged_df['campaign_id'] == campaign_id), 'revenue'].sum()
        clicks_per_bnr = merged_df.loc[(merged_df['banner_id'] == bnr_ids) & (
            merged_df['campaign_id'] == campaign_id), 'click_id'].count()
        banner_rev_dict[bnr_ids] = revenues_per_bnr
        banner_clicks_dict[bnr_ids] = clicks_per_bnr
    return banner_rev_dict, banner_clicks_dict

merged_df = load_csvs(datetime.datetime.now(), file_path)
#find_banners_with_conv(27, merged_df)

In [198]:
def create_banner_campaign_dataframe(merged_df):
    start = time.time()
    campaigns = impressions_df['campaign_id'].unique().tolist()
    #resultant_df = pd.DataFrame(columns=('campaign_id', 'list_of_banners'))
    for cmpgn in campaigns:
        revenue_dict, click_dict = find_banners_with_conv(cmpgn, merged_df)
        sorted_revenue_dict = {k: v for k,v in sorted(revenue_dict.items(), key=itemgetter(1), reverse=True)}
        sorted_click_dict = {k: v for k,v in sorted(click_dict.items(), key=itemgetter(1), reverse=True)}
        #print (sorted_revenue_dict, sorted_click_dict)
        no_of_banners_with_conv = sum(value > 0 for value in sorted_revenue_dict.values())
        sum_of_revenue_per_bnr = sum(sorted_revenue_dict.values())
        #print (no_of_banners_with_conv)
        if no_of_banners_with_conv >= 10:
            output_banners = list(sorted_revenue_dict.keys())[:10]
        elif no_of_banners_with_conv > 5 and no_of_banners_with_conv < 10:
            output_banners = list(sorted_revenue_dict.keys())
        elif no_of_banners_with_conv > 0 and no_of_banners_with_conv <= 5:
            output_banners = list(sorted_revenue_dict.keys())
        else:
            output_banners = [x for x in sorted_click_dict if sorted_click_dict[x] < 5]
        print (cmpgn, output_banners, sum_of_revenue_per_bnr)
        elapsed_time = time.time() - start
        print (elapsed_time)
        
    
merged_df = load_csvs(datetime.datetime.now(), file_path)  
create_banner_campaign_dataframe(merged_df)

27 [390, 113, 406, 484, 171, 204, 314, 147, 403, 102] 59.10224300000001
0.5829708576202393
25 [399, 200, 196, 296, 363, 162, 286, 127, 381, 236] 54.783939999999994
1.1614336967468262
49 [242, 229, 234, 139, 260, 161, 165, 231, 226, 405] 61.666976999999996
1.7394979000091553
18 [458, 355, 346, 461, 341, 500, 266, 170, 337, 297] 63.03330399999999
2.487959623336792
19 [166, 160, 228, 103, 345, 375, 178, 213, 306, 325] 57.699197999999996
3.1623668670654297
12 [381, 387, 206, 297, 295, 319, 194, 275, 424, 142] 47.272997999999994
3.8412349224090576
13 [132, 236, 153, 444, 166, 292, 324, 362, 400, 432] 40.775151
4.605071783065796
40 [483, 152, 277, 335, 466, 134, 368, 290, 360, 438] 52.49319
5.247909784317017
38 [377, 402, 415, 486, 441, 382, 212, 245, 206, 231] 60.941969000000014
5.883480787277222
50 [447, 306, 322, 140, 445, 479, 320, 215, 485, 393] 48.994769999999995
6.674692869186401
4 [230, 213, 326, 451, 423, 134, 358, 234, 255, 292] 60.611380999999994
7.397143602371216
28 [417, 301, 14